In [1]:
import os
import json
import requests
from tqdm.auto import tqdm

In [2]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) # suppress requests warnings of InsecureRequestWarning

# Exploring Allam APIs

In [3]:
ACCESS_TOKEN = 'xdv6hDyIRvQapMeJ8WjKkoYA9787nkqdXq358jkh37iUlVnHhrHeBP10l3Fa7hwA3PJjlaZZeDXNe5W6DCXII0Sgkw78dDq7gzbM6vjuKgFNVNjWAiSPsYaYOMAGAm56'

In [4]:
def get_chat_completion(messages):
    payload = {
        "messages": messages,
        #"temperature": 0.6,
        "temperature": 0.0,
        "stream": False,
        "model": "allam",
        #"top_p": 0.98,
        "n": 1,
        "add_generation_prompt": True,
        "echo": False,
        "stop": " </s>",
    }
    response = requests.post(
        'https://vllm-v19.allam.ai/v1/chat/completions',
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {ACCESS_TOKEN}"
        },
        data=json.dumps(payload),
        timeout=150,
        verify=False,
    )
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON
        chat_response_data = response.json()
        # print(chat_response_data)
        # return chat_response_data['choices'][0]['message']['content']
        return chat_response_data
    else:
        print(f"Request failed with status code {response.status_code}")
        print(response.text)
        return response

test the API

In [5]:
print(get_chat_completion([
    {
        'role': 'user',
        'content': 'ما هو لون نجوم السماء؟'
    },
]))

{'id': 'cmpl-615ba42f5aa24d5089b2985f85464d34', 'object': 'chat.completion', 'created': 1732443028, 'model': 'allam', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': ' لون النجوم في السماء يعتمد على درجة حرارة سطح النجم. النجوم الأكثر سخونة تظهر باللون الأزرق، بينما النجوم الأقل حرارة تظهر باللون الأحمر. النجوم الصفراء هي الأكثر شيوعاً وتمثل النجوم ذات درجات حرارة سطح متوسطة. إذا نظرت إلى السماء ليلاً، سترى مجموعة متنوعة من الألوان بسبب درجات الحرارة المختلفة للنجوم. '}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None}], 'usage': {'prompt_tokens': 15, 'total_tokens': 78, 'completion_tokens': 63}}


# Generating the dataset

first, we generate articles by polishing them.

In [6]:
def save_generated_posts_to_jsonl(posts, file_path):  
    # Append new articles
    with open(file_path, 'w', encoding='utf-8') as f:
        for post in posts:
            json_object = post
            f.write(json.dumps(json_object, ensure_ascii=False) + '\n')

def load_existing_posts(file_path):
    posts = []
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                posts.append(json.loads(line))
    return posts

In [7]:
def load_posts():
    posts = []
    with open('arabic_datasets/social_media_dataset.json', 'r', encoding='utf-8') as f:
        posts = json.load(f)
    return posts

In [8]:
posts = load_posts()
len(posts)

3500

In [9]:
# Your existing code to generate articles
def generate_posts(generation_prompts, generation_type):
    
    # Define file path for saving posts
    os.makedirs("generated_arabic_datasets/allam/arabic_social_media_dataset", exist_ok=True)
    file_path = f"generated_arabic_datasets/allam/arabic_social_media_dataset/{generation_type}_posts_generation.jsonl"
    generated_posts = load_existing_posts(file_path)

    for i, prompt in tqdm(
        enumerate(
            generation_prompts[len(generated_posts) :],
            start=len(generated_posts),
        ),
        total=len(generation_prompts),
        initial=len(generated_posts)
    ):
        output_post = {}
        generated_chat = get_chat_completion(
            [
                {
                    "role": "user",
                    "content": prompt,
                },
            ]
        )
        if not isinstance(generated_chat, dict): # in this case, the  generated_chat is the request's Response object
            generated_post = ''
            # catch the case of errors other than the context length
            if not generated_chat.json()['message'].startswith("This model's maximum context length is 4096 tokens. However, you requested"):
                raise
        else:
            generated_post = generated_chat['choices'][0]['message']['content']
        
        # do some preprocessing
        generated_post = generated_post.strip().strip('<END>').strip().strip('<START>').strip()
        end_token_index = generated_post.find('<END>')
        if end_token_index > 0:
            generated_post = generated_post[:end_token_index]
        generated_post = generated_post.strip()
        
        output_post["original_post"] = posts[i]
        output_post["generated_post"] = generated_post
        generated_posts.append(output_post)
        # Save the generated posts
        save_generated_posts_to_jsonl(generated_posts, file_path)

## Generate by polishing posts

In [10]:
generate_by_polishing_post_prompt = """
أنت خبير في تحليل وتحسين منشورات وسائل التواصل الاجتماعي. مهمتك هي تحسين وصقل المنشور المقدم مع الحفاظ على أسلوبه وطابعه الأصلي.

المطلوب تحسين المنشور مع مراعاة:
- الحفاظ على اللهجة المستخدمة إن وجدت
- الإبقاء على التشكيل إن وجد
- استخدام مفردات قريبة من المفردات الأصلية
- تصحيح الأخطاء الإملائية والنحوية دون تغيير جوهري في الأسلوب
- تحسين التماسك والوضوح مع الحفاظ على الطابع العفوي للمنشور
- الحفاظ على عدد كلمات مقارب للمنشور الأصلي ما أمكن وحيثما كان مناسباً

مهم جداً: يجب الالتزام بالتالي بدقة:
- ابدأ المنشور المحسّن مباشرة بعد علامة <START>
- انتهِ المنشور المحسّن بعلامة <END>
- لا تضف أي تعليقات أو ملاحظات قبل <START> أو بعد <END>
- لا تكتب أي عبارات تمهيدية أو ختامية
- لا تضف أي نص ليس جزءاً من المنشور المحسّن نفسه
- لا تعلق على التحسينات أو التغييرات التي قمت بها

المنشور الأصلي:
{post}

المنشور المحسّن:
<START>
""".strip()

In [11]:
print(generate_by_polishing_post_prompt.format(post=posts[0]))

أنت خبير في تحليل وتحسين منشورات وسائل التواصل الاجتماعي. مهمتك هي تحسين وصقل المنشور المقدم مع الحفاظ على أسلوبه وطابعه الأصلي.

المطلوب تحسين المنشور مع مراعاة:
- الحفاظ على اللهجة المستخدمة إن وجدت
- الإبقاء على التشكيل إن وجد
- استخدام مفردات قريبة من المفردات الأصلية
- تصحيح الأخطاء الإملائية والنحوية دون تغيير جوهري في الأسلوب
- تحسين التماسك والوضوح مع الحفاظ على الطابع العفوي للمنشور
- الحفاظ على عدد كلمات مقارب للمنشور الأصلي ما أمكن وحيثما كان مناسباً

مهم جداً: يجب الالتزام بالتالي بدقة:
- ابدأ المنشور المحسّن مباشرة بعد علامة <START>
- انتهِ المنشور المحسّن بعلامة <END>
- لا تضف أي تعليقات أو ملاحظات قبل <START> أو بعد <END>
- لا تكتب أي عبارات تمهيدية أو ختامية
- لا تضف أي نص ليس جزءاً من المنشور المحسّن نفسه
- لا تعلق على التحسينات أو التغييرات التي قمت بها

المنشور الأصلي:
هنا الكثير من منيف، والكثير من شرق المتوسط أيضا. الأم هنا متعب الهذال في مدن الملح، أما أنيسة ورجب وحامد فهم الإنسان بكل تقلباته. لم أحب شخصية أنيسة، أزعجتني حقيقة. الرواية على أنها رائعة، وعلى أنها ذك

In [12]:
generate_by_polishing_posts_prompts = []
for post in posts:
    prompt = generate_by_polishing_post_prompt.format(post=post)
    generate_by_polishing_posts_prompts.append(prompt)

In [13]:
len(generate_by_polishing_posts_prompts)

3500

In [14]:
generate_by_polishing_posts_prompts[:3]

["أنت خبير في تحليل وتحسين منشورات وسائل التواصل الاجتماعي. مهمتك هي تحسين وصقل المنشور المقدم مع الحفاظ على أسلوبه وطابعه الأصلي.\n\nالمطلوب تحسين المنشور مع مراعاة:\n- الحفاظ على اللهجة المستخدمة إن وجدت\n- الإبقاء على التشكيل إن وجد\n- استخدام مفردات قريبة من المفردات الأصلية\n- تصحيح الأخطاء الإملائية والنحوية دون تغيير جوهري في الأسلوب\n- تحسين التماسك والوضوح مع الحفاظ على الطابع العفوي للمنشور\n- الحفاظ على عدد كلمات مقارب للمنشور الأصلي ما أمكن وحيثما كان مناسباً\n\nمهم جداً: يجب الالتزام بالتالي بدقة:\n- ابدأ المنشور المحسّن مباشرة بعد علامة <START>\n- انتهِ المنشور المحسّن بعلامة <END>\n- لا تضف أي تعليقات أو ملاحظات قبل <START> أو بعد <END>\n- لا تكتب أي عبارات تمهيدية أو ختامية\n- لا تضف أي نص ليس جزءاً من المنشور المحسّن نفسه\n- لا تعلق على التحسينات أو التغييرات التي قمت بها\n\nالمنشور الأصلي:\nهنا الكثير من منيف، والكثير من شرق المتوسط أيضا. الأم هنا متعب الهذال في مدن الملح، أما أنيسة ورجب وحامد فهم الإنسان بكل تقلباته. لم أحب شخصية أنيسة، أزعجتني حقيقة. الرواية على أنه

In [15]:
generate_posts(generation_prompts=generate_by_polishing_posts_prompts, generation_type='by_polishing')

 92%|#########1| 3215/3500 [00:00<?, ?it/s]